In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
def calc(N): #no. of ways to exchange the currencies
    sumfac = 0
    for i in range(N-1):
        sumfac += math.factorial(N-2) / math.factorial(i)
    return sumfac

In [3]:
calc(4)

5.0

### Question 1 ###

### Question 2 ###

In [5]:
def exists(v):
    if 'v' in locals():
        return True
    elif 'v' in globals():
        return True
    else:
        return False
  # myVar exists.

In [6]:
exists(b)

NameError: name 'b' is not defined

In [7]:
a = 1

### Question 3 ###

In [11]:
def pascal(N):
    pascal = [[1], [1,1]]
    if N == 1:
        print(1)
    else:
        for i in range(1,N-1):
            ans = [1]
            for j in range(len(pascal[i])-1):
                ans.append(pascal[i][j] + pascal[i][j+1])
            ans.append(1)
            pascal.append(ans)
        for i in pascal:
            print(*i, sep = " ") 

# def pascal_triangle(N):
#     def nextline(l):
#         ans = [1]
#         for i in range(len(l)-1):
#             ans.append(l[i] + l[i+1])
#         ans.append(1)
#         #print(*ans, sep=" ")
#         return ans
#     pascal = [[1], [1,1]]
#     if N == 1:
#         print(1)
#     else:
#         for i in range(1,N-1):
#             pascal.append(nextline(pascal[i]))
#         for i in pascal:
#             print(*i,sep=" ")

### Question 4 ###

### Question 5 ###

In [12]:
#How many python files
git ls-files "./*.py" | wc -l

SyntaxError: invalid syntax (Temp/ipykernel_3912/2845812811.py, line 2)

### Question 6 ###